In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import re
import requests

# Extrakce dat z webové stránky do tabulky

## Příklad

Extrakce dat o 50 nejpopulárnějších knihách ze žánru beletrie.

### Cíl

Vytvořit tabulku s následujícími sloupci:

- Název knihy
- Průměrné hodnocení
- Celkový počet hodnocení
- Datum vydání

In [2]:
url = 'https://www.goodreads.com/shelf/show/fiction'
r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')

In [3]:
book_title = [
    item.text for item in soup.find_all(attrs={'class':'bookTitle'})
]
book_title

['To Kill a Mockingbird (To Kill a Mockingbird, #1)',
 '1984 (Kindle Edition)',
 'The Great Gatsby (Paperback)',
 "Harry Potter and the Sorcerer's Stone (Harry Potter, #1)",
 'Animal Farm (Mass Market Paperback)',
 'The Catcher in the Rye (Paperback)',
 'The Hunger Games (The Hunger Games, #1)',
 'The Da Vinci Code (Robert Langdon, #2)',
 'The Kite Runner (Paperback)',
 "The Handmaid's Tale (Paperback)",
 'Lord of the Flies (Paperback)',
 'Harry Potter and the Chamber of Secrets (Harry Potter, #2)',
 'Pride and Prejudice (Paperback)',
 'The Hobbit or There and Back Again (Paperback)',
 'Harry Potter and the Prisoner of Azkaban (Harry Potter, #3)',
 'The Alchemist (Paperback)',
 'Harry Potter and the Goblet of Fire (Harry Potter, #4)',
 'Life of Pi (Paperback)',
 'Catching Fire (The Hunger Games, #2)',
 'Fahrenheit 451 (Kindle Edition)',
 'Harry Potter and the Deathly Hallows (Harry Potter, #7)',
 'The Girl with the Dragon Tattoo (Millennium, #1)',
 'Harry Potter and the Half-Blood Prin

In [4]:
year = [
    int(
        re.search(
            "published (\d{4})", 
            item.text.split('—\n')[2]
        ).group(1)
    ) for item in soup.find_all(attrs={'class':'greyText smallText'})]
year

[1960,
 1949,
 1926,
 1997,
 1945,
 1951,
 2008,
 2003,
 2003,
 1985,
 1954,
 1998,
 1813,
 1937,
 1999,
 1988,
 2000,
 2001,
 2009,
 1953,
 2007,
 2005,
 2005,
 2003,
 1932,
 2010,
 2005,
 2003,
 2009,
 1937,
 2012,
 2000,
 1969,
 1847,
 2012,
 2003,
 2002,
 1996,
 1997,
 2006,
 2006,
 2015,
 1979,
 1961,
 1884,
 1967,
 1954,
 1818,
 1890,
 2014]

## Odbočka: Regulární výrazy

Co znamená `re.findall("published (\d{4})", nejaky_text)` se dozvíte v extra [prezentaci](https://sedlakovi.github.io/webscraping/6a_regex.slides.html).

In [5]:
one_book = soup.find(attrs={'class':'greyText smallText'}).text
one_book

'\n                avg rating 4.27 —\n                4,005,466 ratings  —\n                published 1960\n              '

In [6]:
# Jde to i jen regexem ;)
match = re.search ('avg rating (\d+\.\d+).+ (\d{1,3}(,\d{3})*).+ published (\d{4})', one_book, re.DOTALL)
print('average rating', match.group(1))
print('total number of ratings', match.group(2).replace(',', ''))
print('year published', match.group(4))

average rating 4.27
total number of ratings 4005466
year published 1960


In [7]:
avg_rating = [
    float(
        re.search("\d+(\.\d+)?", item.text.split('—\n')[0]).group(0)
    ) for item in soup.find_all(attrs={'class':'greyText smallText'})
]
avg_rating

[4.27,
 4.17,
 3.91,
 4.47,
 3.92,
 3.8,
 4.33,
 3.83,
 4.29,
 4.1,
 3.67,
 4.41,
 4.25,
 4.27,
 4.56,
 3.85,
 4.55,
 3.9,
 4.29,
 3.98,
 4.62,
 4.13,
 4.56,
 4.49,
 3.98,
 4.03,
 4.37,
 3.87,
 4.46,
 3.86,
 4.06,
 3.89,
 4.07,
 4.12,
 4.23,
 3.96,
 3.8,
 4.45,
 4.1,
 4.08,
 3.96,
 3.91,
 4.22,
 3.98,
 3.81,
 4.06,
 4.35,
 3.78,
 4.07,
 4.33]

In [8]:
total_ratings = [
    int(
        re.search(
            "(\d{1,3}(,\d{3})*)", 
            item.text.split('—\n')[1]
        ).group(1).replace(',', '')) 
    for item in soup.find_all(attrs={'class':'greyText smallText'})]
total_ratings

[4005466,
 2662838,
 3342907,
 6080900,
 2396393,
 2464757,
 5796273,
 1743300,
 2195449,
 1162443,
 2024675,
 2350786,
 2625608,
 2568490,
 2395757,
 1797825,
 2242177,
 1211687,
 2225161,
 1450591,
 2414777,
 2327809,
 2137088,
 2190514,
 1284279,
 2094209,
 1608698,
 1076272,
 1909231,
 1752569,
 2020964,
 2417176,
 1031816,
 1464578,
 3093784,
 1459913,
 1855760,
 1794998,
 1587334,
 1262666,
 649375,
 1763902,
 1255963,
 670032,
 1085937,
 671673,
 2131165,
 1038524,
 835335,
 833659]

In [9]:
books = pd.DataFrame(
    zip(book_title, avg_rating, total_ratings, year), 
    columns=['Title', 'Average ratings', 'Total ratings', 'Year published']
)
books

,Title,Average ratings,Total ratings,Year published
0,"To Kill a Mockingbird (To Kill a Mockingbird, #1)",4.27,4005466,1960
1,1984 (Kindle Edition),4.17,2662838,1949
2,The Great Gatsby (Paperback),3.91,3342907,1926
3,Harry Potter and the Sorcerer's Stone (Harry P...,4.47,6080900,1997
4,Animal Farm (Mass Market Paperback),3.92,2396393,1945
5,The Catcher in the Rye (Paperback),3.80,2464757,1951
6,"The Hunger Games (The Hunger Games, #1)",4.33,5796273,2008
7,"The Da Vinci Code (Robert Langdon, #2)",3.83,1743300,2003
8,The Kite Runner (Paperback),4.29,2195449,2003
9,The Handmaid's Tale (Paperback),4.10,1162443,1985


Pomocí metody `round` s parametrem `decimals=-1` můžeme zaokrouhlit rok vydání na dekády.

In [10]:
books['Year published rounded'] = books['Year published'].round(
    decimals=-1
)
books

,Title,Average ratings,Total ratings,Year published,Year published rounded
0,"To Kill a Mockingbird (To Kill a Mockingbird, #1)",4.27,4005466,1960,1960
1,1984 (Kindle Edition),4.17,2662838,1949,1950
2,The Great Gatsby (Paperback),3.91,3342907,1926,1930
3,Harry Potter and the Sorcerer's Stone (Harry P...,4.47,6080900,1997,2000
4,Animal Farm (Mass Market Paperback),3.92,2396393,1945,1940
5,The Catcher in the Rye (Paperback),3.80,2464757,1951,1950
6,"The Hunger Games (The Hunger Games, #1)",4.33,5796273,2008,2010
7,"The Da Vinci Code (Robert Langdon, #2)",3.83,1743300,2003,2000
8,The Kite Runner (Paperback),4.29,2195449,2003,2000
9,The Handmaid's Tale (Paperback),4.10,1162443,1985,1980


Toto zaokrouhlení se ale nehodí pro dekády. Níže jsou uvedeny 2 způsoby jak se dá vytvořit sloupec s dekádou.

In [11]:
import numpy as np
(np.floor((books['Year published'] / 10)) *10).astype(int)

0     1960
1     1940
2     1920
3     1990
4     1940
5     1950
6     2000
7     2000
8     2000
9     1980
10    1950
11    1990
12    1810
13    1930
14    1990
15    1980
16    2000
17    2000
18    2000
19    1950
20    2000
21    2000
22    2000
23    2000
24    1930
25    2010
26    2000
27    2000
28    2000
29    1930
30    2010
31    2000
32    1960
33    1840
34    2010
35    2000
36    2000
37    1990
38    1990
39    2000
40    2000
41    2010
42    1970
43    1960
44    1880
45    1960
46    1950
47    1810
48    1890
49    2010
Name: Year published, dtype: int64

In [12]:
books['Decade'] = books['Year published'].astype(str).str.replace(
    '(\d{3})\d', '\g<1>0', regex=True
).astype(int)

In [13]:
books.groupby('Decade').size()

Decade
1810     2
1840     1
1880     1
1890     1
1920     1
1930     3
1940     2
1950     4
1960     4
1970     1
1980     2
1990     5
2000    18
2010     5
dtype: int64

## Malá odbočka - filtrování tabulky

```
data[podmínka]

data[(podmínka1) & (podmínka2)]
```

Pro kombinování podmínek lze použit následující operatory:
- `&` zároveň
- `|` nebo
- `~` ne

In [14]:
# Vybereme knihy, vydané před rokem 2000 s průměrným hodnocením více než 4.0
books[(books['Year published'] < 2000) & (books['Average ratings'] > 4.0)]

,Title,Average ratings,Total ratings,Year published,Year published rounded,Decade
0,"To Kill a Mockingbird (To Kill a Mockingbird, #1)",4.27,4005466,1960,1960,1960
1,1984 (Kindle Edition),4.17,2662838,1949,1950,1940
3,Harry Potter and the Sorcerer's Stone (Harry P...,4.47,6080900,1997,2000,1990
9,The Handmaid's Tale (Paperback),4.10,1162443,1985,1980,1980
11,Harry Potter and the Chamber of Secrets (Harry...,4.41,2350786,1998,2000,1990
12,Pride and Prejudice (Paperback),4.25,2625608,1813,1810,1810
13,The Hobbit or There and Back Again (Paperback),4.27,2568490,1937,1940,1930
14,Harry Potter and the Prisoner of Azkaban (Harr...,4.56,2395757,1999,2000,1990
32,Slaughterhouse-Five (Paperback),4.07,1031816,1969,1970,1960
33,Jane Eyre (Paperback),4.12,1464578,1847,1850,1840


# Cvičení 6.1

1. Ze stránek s mobilními telefony na heurece [https://mobilni-telefony.heureka.cz/](https://mobilni-telefony.heureka.cz/) vytvořte tabulku s následujícími sloupci:
    - Název telefona
    - Hodnocení
    - Počet recenzí
    - Minimální cena
    - Maximální cena
2. Vytvořte sloupec, kde zaokrouhlíte minimální cenu na celé tisíce
3. Seskupte telefony dle nově vytvořeného sloupce a zjistěte průměrné hodnocení pro každou skupinu

# Extrakce HTML kódu do tabulky

Lze použit funkci `read_html`, která vyžaduje HTML řetězec (objekt `BeautifulSoup` musí být převeden na `str`).

In [15]:
url = 'https://www.czso.cz/csu/czso/aktualniinformace'
r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')

In [16]:
soup.find(class_="table nu-table")

<table class="table nu-table" style="padding:0.1em; width: 650px; border:1px solid #184D84; margin-left:0.35em;">
<!--****HLAVICKA*****-->
<tbody>
<tr style="background-color:#0D7ABC">
<td style="text-align: center; vertical-align: middle; width: 48%;">
<span style="color: #FFFFFF;"><strong>Ukazatel</strong></span></td>
<td style="text-align: center; vertical-align: middle; width: 18%;">
<span style="color: #FFFFFF;"><strong>Období</strong></span></td>
<td align="center" valign="top" width="18%">
<span style="color: #FFFFFF;"><strong>Meziroční<br/>
					růst/pokles<br/>
					(v %)</strong></span></td>
<td style="text-align: center; vertical-align: middle; width: 18%;">
<span style="color: #FFFFFF;"><strong>Datum<br/>
					zveřejnění</strong></span></td>
</tr>
<!--******** HDP ******-->
<tr style="background-color:#F1F0F0">
<td valign="bottom">
<a href="#11">Hrubý domácí produkt</a></td>
<td align="center" valign="bottom">
					2. čtvrtletí 2019</td>
<td align="center" valign="bottom">


In [17]:
pd.read_html(str(soup.find(class_="table nu-table")), header=0)[0]

,Ukazatel,Období,Meziroční růst/pokles (v %),Datum zveřejnění
0,Hrubý domácí produkt,2. čtvrtletí 2019,27,14.08.2019
1,Index spotřebitelských cen,červenec 2019,29,12.08.2019
2,Míra inflace,červenec 2019,26,12.08.2019
3,Průmyslová produkce,červen 2019,-38,06.08.2019
4,Stavební produkce,červen 2019,24,06.08.2019
5,Tržby v maloobchodě (CZ-NACE 47),červen 2019,46,07.08.2019
6,Průměrná mzda - nominální,1. čtvrtletí 2019,74,04.06.2019
7,Průměrná mzda - reálná,1. čtvrtletí 2019,46,04.06.2019
8,Indexy cen výrobců - zemědělských,červenec 2019,121,16.08.2019
9,Indexy cen výrobců - průmyslových,červenec 2019,21,16.08.2019


In [18]:
economic_stat = pd.read_html(str(soup.find(class_="table nu-table")), header=0)[0]
economic_stat

,Ukazatel,Období,Meziroční růst/pokles (v %),Datum zveřejnění
0,Hrubý domácí produkt,2. čtvrtletí 2019,27,14.08.2019
1,Index spotřebitelských cen,červenec 2019,29,12.08.2019
2,Míra inflace,červenec 2019,26,12.08.2019
3,Průmyslová produkce,červen 2019,-38,06.08.2019
4,Stavební produkce,červen 2019,24,06.08.2019
5,Tržby v maloobchodě (CZ-NACE 47),červen 2019,46,07.08.2019
6,Průměrná mzda - nominální,1. čtvrtletí 2019,74,04.06.2019
7,Průměrná mzda - reálná,1. čtvrtletí 2019,46,04.06.2019
8,Indexy cen výrobců - zemědělských,červenec 2019,121,16.08.2019
9,Indexy cen výrobců - průmyslových,červenec 2019,21,16.08.2019


In [19]:
economic_stat.tail(1).index

RangeIndex(start=16, stop=17, step=1)

In [20]:
economic_stat.drop(economic_stat.tail(1).index, inplace=True)

In [21]:
economic_stat

,Ukazatel,Období,Meziroční růst/pokles (v %),Datum zveřejnění
0,Hrubý domácí produkt,2. čtvrtletí 2019,27,14.08.2019
1,Index spotřebitelských cen,červenec 2019,29,12.08.2019
2,Míra inflace,červenec 2019,26,12.08.2019
3,Průmyslová produkce,červen 2019,-38,06.08.2019
4,Stavební produkce,červen 2019,24,06.08.2019
5,Tržby v maloobchodě (CZ-NACE 47),červen 2019,46,07.08.2019
6,Průměrná mzda - nominální,1. čtvrtletí 2019,74,04.06.2019
7,Průměrná mzda - reálná,1. čtvrtletí 2019,46,04.06.2019
8,Indexy cen výrobců - zemědělských,červenec 2019,121,16.08.2019
9,Indexy cen výrobců - průmyslových,červenec 2019,21,16.08.2019


# Cvičení 6.2

Ze stránek [ministersva financí](https://www.mfcr.cz/cs/verejny-sektor/statni-rozpocet/legislativa-statniho-rozpoctu/2019/rozpocet-resortu-financi-kapitola-312-20-33908) vyextrahujte tabulku rozpočtu.